#Engenharia de Dados - Camada Silver (Padronização e Limpeza)

**Objetivo**
Transformar os dados brutos da camada Bronze em dados confiáveis e padronizados na camada Silver

**Principais Transformações**

1. **Renomeação:** Tradução de colunas (Inglês -> Português) e padronização para *snake_case*

2. **Tipagem:** Conversão explícita de tipos (ex: String -> Timestamp/Double)

3. **Sanitização:**
  * **Cidades:** Remoção de acentos e conversão para minúsculo
  * **Reviews:** Tratamento de caracteres especias (quebras de linha) e nulos

4.  **Persistência:** Gravação em formato Delta Lake gerenciado pelo Unity Catalog.

# Importando bibliotecas e definindo variáveis para caminho

In [0]:
from pyspark.sql.functions import col, trim, lower, regexp_replace, when, lit
from pyspark.sql.types import DoubleType, IntegerType

In [0]:
catalog = "workspace_ecommerce"
schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
print(f"Configuração ativada: \nCatalog: {catalog}\nOrigem: {schema_bronze}\nDestino: {schema_silver}")

Configuração ativada: 
Catalog: workspace_ecommerce
Origem: bronze
Destino: silver


##1.0 Mapeamento de Metadados (De/Para)

Para garantir escalabilidade e organização, utilizaremos um dicionário de configuração ('schema_map').

Este objeto define, para cada tabela, quais colunas devem ser selecionadas e qual será seu novo nome na camada Silver.

* **Chave:** Nome da tabela original.

* **Valor:** Dicionário onde 'chave = coluna_original' e 'valor = nova_coluna_pt'.

In [0]:
#Mapeamento de Tradução e Seleção de Colunas
schema_map = {
    "customers": {
        'customer_id': 'id_cliente',
        'customer_unique_id': 'id_cliente_unico',
        'customer_zip_code_prefix': 'nr_cep_prefixo',
        'customer_city': 'nm_cidade',
        'customer_state': 'sg_estado'
    },
    "geolocation": {
        'geolocation_zip_code_prefix': 'nr_cep_prefixo',
        'geolocation_lat': 'nr_latitude',
        'geolocation_lng': 'nr_longitude',
        'geolocation_city': 'nm_cidade',
        'geolocation_state': 'sg_estado'
    },
    "order_items": {
        'order_id': 'id_pedido',
        'order_item_id': 'nr_item_pedido',
        'product_id': 'id_produto',
        'seller_id': 'id_vendedor',
        'shipping_limit_date': 'dt_limite_envio',
        'price': 'vl_preco',
        'freight_value': 'vl_frete'
    },
    "order_payments": {
        'order_id': 'id_pedido',
        'payment_sequential': 'nr_sequencia_pagamento',
        'payment_type': 'desc_tipo_pagamento',
        'payment_installments': 'nr_parcelas',
        'payment_value': 'vl_pagamento'
    },
    "order_reviews": {
        'review_id': 'id_avaliacao',
        'order_id': 'id_pedido',
        'review_score': 'vl_nota',
        'review_comment_title': 'desc_titulo_comentario',
        'review_comment_message': 'desc_comentario',
        'review_creation_date': 'dt_envio_pesquisa',
        'review_answer_timestamp': 'dt_resposta_pesquisa'
    },
    "orders": {
        'order_id': 'id_pedido',
        'customer_id': 'id_cliente',
        'order_status': 'desc_situacao',
        'order_purchase_timestamp': 'dt_compra',
        'order_approved_at': 'dt_aprovacao',
        'order_delivered_carrier_date': 'dt_envio_transportadora',
        'order_delivered_customer_date': 'dt_entrega_cliente',
        'order_estimated_delivery_date': 'dt_estimada_entrega'
    },
    "product_category_name_translation": {
        'product_category_name': 'desc_categoria_pt',
        'product_category_name_english': 'desc_categoria_en'
    },
    "products": {
        'product_id': 'id_produto',
        'product_category_name': 'desc_categoria',
        'product_name_lenght': 'nr_tamanho_nome',
        'product_description_lenght': 'nr_tamanho_desc',
        'product_photos_qty': 'nr_qtd_fotos',
        'product_weight_g': 'nr_peso_gramas',
        'product_length_cm': 'nr_comprimento_cm',
        'product_height_cm': 'nr_altura_cm',
        'product_width_cm': 'nr_largura_cm'
    },
    "sellers": {
        'seller_id': 'id_vendedor',
        'seller_zip_code_prefix': 'nr_cep_prefixo',
        'seller_city': 'nm_cidade',
        'seller_state': 'sg_estado'
    }
}

## 2.0 Motor de Transformação (ETL)

A função 'processar_silver' encapsula a lógica de negócio. Ela itera sobre o mapeamento definido acima e aplica as seguintes regras:

1. **Normalização de Cidades:** Aplica 'lower()', 'trim()', e remove acentuações via Regex para corrigir inconsistências (ex: "São Paulo" -> "Sao Paulo")

2. **Limpeza de Texto:** Remove quebras de linhas ('\n') que quebram exportações CSV e preenche valores nulos com "nao_informado".

3. **Linhagem:** Mantém as colunas de metadados 'dt_ingestao' e 'arquivo_origem' para rastreabilidade

In [0]:
def processar_silver(nome_tabela, mapeamento):
    print(f"Iniciando processamento: {nome_tabela}")


    try:
        # 1. Leitura camada Bronze
        df = spark.read.table(f"{catalog}.{schema_bronze}.{nome_tabela}")
        colunas_disponiveis = df.columns

        colunas_finais = []

        # 2. Aplicação das regras de Negócio Coluna a Coluna
        for col_origem, col_nova in mapeamento.items():

            # TRAVA DE SEGURANÇA
            if col_origem not in colunas_disponiveis:
                print(f"AVISO: Coluna '{col_origem}' não encontrada na origem. Pulando mapeamento para '{col_nova}'.")
                continue #Pula para a próxima coluna sem quebrar

            col_transf = col(col_origem)

            # REGRAS DE TIPAGEM (CAST INTELIGENTE)
            #Timestamp (dt_)
            if col_nova.startswith("dt_"):
                 col_transf = col_transf.cast("timestamp")

            #Double/Moeda (vl_)
            elif col_nova.startswith("vl_"):
                col_transf = col_transf.cast("double")

            #Regra para CEP virar String
            elif "cep" in col_nova:
                col_transf = col_transf.cast("string")

            #Inteiro (nr_), exceto lat/long
            elif col_nova.startswith("nr_") and "latitude" not in col_nova and "longitude" not in col_nova:
                col_transf = col_transf.cast("int")

            # REGRA: Normalização de Texto (Cidades)
            if col_origem in ["customer_city", "geolocation_city", "seller_city", "nm_cidade"]:
                # Remove Acentos: á -> a, é -> e, etc
                col_transf = trim(lower(regexp_replace(col_transf, "[áàâãä]", "a")))
                col_transf = trim(lower(regexp_replace(col_transf, "[éèêë]", "e")))
                col_transf = trim(lower(regexp_replace(col_transf, "[íìîï]", "i")))
                col_transf = trim(lower(regexp_replace(col_transf, "[óòôõö]", "o")))
                col_transf = trim(lower(regexp_replace(col_transf, "[úùûü]", "u")))
                col_transf = trim(lower(regexp_replace(col_transf, "[ç]", "c")))

            # REGRA: Sanitização de Comentários (Reviews)
            if col_origem in ["review_comment_messages", "review_comment_title"]:
                #Remove quebra de linha (\n, \r)
                col_transf = regexp_replace(col_transf, "[\n, \r]", " ")
                #Trata Nulos
                col_transf = when(col_transf.isNull() | (trim(col_transf) == ""), "nao_informado").otherwise(col_transf)

            # Adiciona a coluna tratada com o novo nome (Alias)
            colunas_finais.append(col_transf.alias(col_nova))

        # 3. Preservação de Metadados (Data Lineage)
        if "dt_ingestao" in df.columns: colunas_finais.append(col("dt_ingestao"))
        if "arquivo_origem" in df.columns: colunas_finais.append(col("arquivo_origem"))

        # 4. Escrita na camada Silver (Delta)
        if len(colunas_finais) > 0:
            df_silver = df.select(*colunas_finais)
            caminho_tabela = f"{catalog}.{schema_silver}.{nome_tabela}"
            df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(caminho_tabela)
            print(f"Tabela criada com sucesso: {caminho_tabela}")
        else:
            print(f"Erro: Nenhuma coluna válida encontrada para a tabela {nome_tabela}")

    except Exception as e:
        print(f"Falha letal ao processar {nome_tabela}: {str(e)}")
        import traceback
        traceback.print_exc()

##3.0 Execução do Pipeline

O bloco abaixo itera sobre todas as tabelas configuradas e executa a transformação em lote.b

In [0]:
#Executa o loop para todas as tabelas mapeadas
for tabela, regras in schema_map.items():
    processar_silver(tabela, regras)

Iniciando processamento: customers
Tabela criada com sucesso: workspace_ecommerce.silver.customers
Iniciando processamento: geolocation
Tabela criada com sucesso: workspace_ecommerce.silver.geolocation
Iniciando processamento: order_items
Tabela criada com sucesso: workspace_ecommerce.silver.order_items
Iniciando processamento: order_payments
Tabela criada com sucesso: workspace_ecommerce.silver.order_payments
Iniciando processamento: order_reviews
Tabela criada com sucesso: workspace_ecommerce.silver.order_reviews
Iniciando processamento: orders
Tabela criada com sucesso: workspace_ecommerce.silver.orders
Iniciando processamento: product_category_name_translation
Tabela criada com sucesso: workspace_ecommerce.silver.product_category_name_translation
Iniciando processamento: products
Tabela criada com sucesso: workspace_ecommerce.silver.products
Iniciando processamento: sellers
Tabela criada com sucesso: workspace_ecommerce.silver.sellers


##4.0 Validação dos Resultados (Data Quality Check)

Abaixo, verificamos uma amostra da tabela 'customers' para confirmar se a normalização de cidades (removendo acentos e maiúsculas) funionou corretamente.

In [0]:
# Verifica se a normalização funcionou (ex: "sao paulo")
display(spark.sql(f"SELECT * FROM {catalog}.{schema_silver}.customers LIMIT 10"))

id_cliente,id_cliente_unico,nr_cep_prefixo,nm_cidade,sg_estado,dt_ingestao,arquivo_origem
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv
4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG,2025-12-30T13:22:59.376193Z,olist_customers_dataset.csv


##5.0 Validação de Tipagem (QA)

O bloco abaixo imprime o Schema final de cada tabela na camada Silver para conferência dos tipos de dados (Timestamps, Doubles, Integers)

In [0]:
#Lista de tabelas que foram processadas na camada Silver
tabelas_silver = list(schema_map.keys())

print(f"Iniciando validação de {len(tabelas_silver)} tabelas na camada Silver...\n")

for tabela in tabelas_silver:
    try:
        #Lê a tabela recém-criada
        df_validacao = spark.read.table(f"{catalog}.{schema_silver}.{tabela}")

        print(f"Tabela: {tabela.upper()}")
        print("-" * 30)

        #O comando printSchema() mostra a árvore de tipos
        df_validacao.printSchema()

        print("\n" + "="*50 + "\n")

    except Exception as e:
        print(f"Erro ao ler {tabela}: {e}")

Iniciando validação de 9 tabelas na camada Silver...

Tabela: CUSTOMERS
------------------------------
root
 |-- id_cliente: string (nullable = true)
 |-- id_cliente_unico: string (nullable = true)
 |-- nr_cep_prefixo: string (nullable = true)
 |-- nm_cidade: string (nullable = true)
 |-- sg_estado: string (nullable = true)
 |-- dt_ingestao: timestamp (nullable = true)
 |-- arquivo_origem: string (nullable = true)



Tabela: GEOLOCATION
------------------------------
root
 |-- nr_cep_prefixo: string (nullable = true)
 |-- nr_latitude: double (nullable = true)
 |-- nr_longitude: double (nullable = true)
 |-- nm_cidade: string (nullable = true)
 |-- sg_estado: string (nullable = true)
 |-- dt_ingestao: timestamp (nullable = true)
 |-- arquivo_origem: string (nullable = true)



Tabela: ORDER_ITEMS
------------------------------
root
 |-- id_pedido: string (nullable = true)
 |-- nr_item_pedido: integer (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- id_vendedor: string (n

##6.0 Alternativa Visual (SQL)

Visual em formato de tabelas mais parecido com Excel, verifica tabelas críticas uma por uma

In [0]:
%sql
 -- DESCRIBE workspace_ecommerce.silver.orders

##7.0 Etapa de verificação de existência de colunas

In [0]:
# SCRIPT DE DIAGNÓSTICO DE COLUNAS
print("Iniciando Varredura de Incosistências...\n")

for tabela, mapping in schema_map.items():
    try:
        #Lê a tabela bronze real 
        df_real = spark.read.table(f"{catalog}.{schema_bronze}.{tabela}")
        cols_reais = df_real.columns

        #Verifica se todas as colunas do mapa existem na tabela
        for col_map in mapping.keys():
            if col_map not in cols_reais:
                print(f"ERRO CRÍTICO NA TABELA '{tabela.upper()}':")
                print(f"   - O dicionário pede a coluna: '{col_map}'")
                print(f"   - Mas as colunas existentes são: {cols_reais}")
                print(f"   -> Dica: Verifique se há erro de digitação (ex: lenght vs length) ou caracteres ocultos.\n")
                
    except Exception as e:
        print(f"❌ Erro ao ler a tabela '{tabela}': {e}\n")

Iniciando Varredura de Incosistências...

